In [5]:
pip install requests

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/b6/7c/8debebb4f90174074b827c63242c23851bdf00a532489fba57fef3416e40/charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for idna<4,>=2.5 from https://files.pythonhosted.org/packages/e5/3e/741d8c82801c347547f8a2a06aa57dbb1992be9e948df2ea0eda2c8b79e8/idna-3.7-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/a2/73/a68704750a7679d0b6d3ad7aa8d4da8e14e151ae82e6fee774e6e0d05ec8/urllib3-2.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/ba/06/a07f096c664aeb9f01624f858c3add0a4e913d6c9625

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import json
import requests
import time

def transcribe_audio(auth_token, audio_file, use_disfluency_filter=False):
    config = {
        "use_multi_channel": False,
        "use_itn": False,
        "use_disfluency_filter": use_disfluency_filter,
        "use_profanity_filter": False,
        "use_paragraph_splitter": True,
        "paragraph_splitter": {
            "max": 50
        }
    }

    resp = requests.post(
        'https://openapi.vito.ai/v1/transcribe',
        headers={'Authorization': f'bearer {auth_token}'},
        data={'config': json.dumps(config)},
        files={'file': open(audio_file, 'rb')}
    )
    resp.raise_for_status()
    task_id = resp.json()['id']
    print(f"작업 ID: {task_id}")

    start_time = time.time()
    timeout = 120  # 2분 제한
    while True:
        resp = requests.get(
            f'https://openapi.vito.ai/v1/transcribe/{task_id}',
            headers={'Authorization': f'bearer {auth_token}'}
        )
        resp.raise_for_status()
        status = resp.json()['status']
        if status == 'completed':
            return resp.json()['results']
        elif status == 'failed':
            raise Exception(f"Transcription failed: {resp.json()['error']}")
        elif time.time() - start_time > timeout:
            raise Exception("Transcription timed out")
        time.sleep(5)

def save_transcription_results(results_without_filter, results_with_filter):
    combined_results = {
        "results_without_filter": results_without_filter,
        "results_with_filter": results_with_filter
    }
    with open("transcription_results.json", "w") as f:
        json.dump(combined_results, f, indent=4, ensure_ascii=False)

# 인증 토큰 받기
resp = requests.post(
    'https://openapi.vito.ai/v1/authenticate',
    data={'client_id': 'pY5fW2sMvQ9qwj5jCHSK',
          'client_secret': '0N05lmQf_kA9oSteVr4X8G6qcv3QaNju7V8x6-zk'}
)
resp.raise_for_status()
auth_token = resp.json()['access_token']

# 음성 파일 전송 및 트랜스크립션 결과 받기
results_without_filter = transcribe_audio(auth_token, 'b.wav', use_disfluency_filter=False)
results_with_filter = transcribe_audio(auth_token, 'B.m4a', use_disfluency_filter=True)

# 트랜스크립션 결과 저장
save_transcription_results(results_without_filter, results_with_filter)


작업 ID: 81Jd41y8QeWTwSv2k1WE4g
작업 ID: qRg4_7RyT42bx4H1J1PUZQ


In [3]:
import json
import requests
import time

def transcribe_audio(auth_token, audio_file, use_disfluency_filter=False):
    config = {
        "use_multi_channel": False,
        "use_itn": False,
        "use_disfluency_filter": use_disfluency_filter,
        "use_profanity_filter": False,
        "use_paragraph_splitter": True,
        "paragraph_splitter": {
            "max": 50
        }
    }

    resp = requests.post(
        'https://openapi.vito.ai/v1/transcribe',
        headers={'Authorization': f'bearer {auth_token}'},
        data={'config': json.dumps(config)},
        files={'file': open(audio_file, 'rb')}
    )
    resp.raise_for_status()
    task_id = resp.json()['id']
    print(f"작업 ID: {task_id}")

    while True:
        resp = requests.get(
            f'https://openapi.vito.ai/v1/transcribe/{task_id}',
            headers={'Authorization': f'bearer {auth_token}'}
        )
        resp.raise_for_status()
        status = resp.json()['status']
        if status == 'completed':
            return resp.json()['results']
        time.sleep(5)

def get_transcription_results(audio_file):
    # 인증 토큰 받기
    resp = requests.post(
        'https://openapi.vito.ai/v1/authenticate',
        data={'client_id': 'pY5fW2sMvQ9qwj5jCHSK',
              'client_secret': '0N05lmQf_kA9oSteVr4X8G6qcv3QaNju7V8x6-zk'}
    )
    resp.raise_for_status()
    auth_token = resp.json()['access_token']

    # 음성 파일 전송 및 트랜스크립션 결과 받기
    results_without_filter = transcribe_audio(auth_token, audio_file, use_disfluency_filter=False)
    results_with_filter = transcribe_audio(auth_token, audio_file, use_disfluency_filter=True)

    return {
        "results_without_filter": results_without_filter,
        "results_with_filter": results_with_filter
    }


In [5]:
# 사용자가 선택한 음성 파일 경로
audio_file = 'b.wav'

# 트랜스크립션 결과 받기
transcription_results = get_transcription_results(audio_file)

def save_transcription_results(transcription_results):
    with open("transcription_results.json", "w") as f:
        json.dump(transcription_results, f, indent=4, ensure_ascii=False)


작업 ID: T5Yoh0rCRjmh1epa1kvURA
작업 ID: TBOC4VpISJOKhDHwUz3pdw
